In [1]:
#!/usr/bin/env python
# coding: utf-8

from argparse import ArgumentParser

In [ ]:
args = ArgumentParser('./build_bwa_scripts.py', description="""This program has been designed to 
    generate bash scripts to align genome sequencing data using BWA, Samtools, and PicardTools. See
    documentation for links to the respective tools. """)

args.add_argument(
	'-i',
	'--input_file',
	help="""
	This is the csv that visualize_cgm.py will read to create the graphs. 
	""",
	default=None
)

args.add_argument(
	'--config_file',
	type=FileType('r'),
	help="This is a config file in JSON format with the information about the databases to be read in. For an example of formatting, see the file LOVD3_Databases.json",
	default="LOVD3_Databases.json",
)

args.add_argument(
	'--disease_gene_lists',
	nargs='+', # This tells the program that if they specify this flag, they have to give it at least one input. If they don't specify it, then the default will go in.
	help="""\
This is a list of text files containing genes associated with the disease of interest. The text files
should each contain a list of gene symbols for one given disease, one gene per line. The name of the disease will be specified
by the arguement --disease_names. """,
	default=["SCID_ny_panel.txt", "Metabolic_diseases_genes.txt"]
)

In [3]:
import json5
with open('bwa_parameters.json', 'r') as cf:
    config_parameters = json5.load(cf)
print(config_parameters['path_to_bwa'])

/home/dwsant/Programs/bwa/bwa


In [4]:
config_parameters

{'read_length': 150,
 'is_paired': True,
 'path_to_bwa': '/home/dwsant/Programs/bwa/bwa',
 'number_cores': 8,
 'path_to_prebuilt_reference': '/home/dwsant/Genome_References/MM39_BWA/MM39',
 'path_to_samtools': '/home/dwsant/Programs/samtools-1.20/samtools',
 'path_to_picard_jar': '/home/dwsant/Programs/picard/build/libs/picard.jar',
 'fai_file': '/home/dwsant/Genome_References/MM39_BWA/Mus_musculus.GRCm39.dna.primary_assembly.fa.fai'}

In [ ]:
import pandas as pd

In [12]:

sample_sheet_2 = pd.read_csv('samples.csv')
sample_sheet_2


,Sample_name,Fastq1,Fastq2
0,TEST,test_r1.fq.gz,test_r2.fq.gz
1,LINE2,test_r1.LINE2.fq.gz,test_r1.LINE2.fq.gz


In [9]:
sample_sheet = pd.read_table('samples.txt', header=None, sep='\t')
sample_sheet

,0,1,2
0,TEST,test_r1.fq.gz,test_r2.fq.gz
1,LINE2,test_r1.LINE2.fq.gz,test_r1.LINE2.fq.gz


In [10]:
for index, row in sample_sheet.iterrows():
    print(row[0])

TEST
LINE2


In [18]:
sample_sheet_single_read = pd.read_csv('samples2.csv')
sample_sheet_single_read

,Sample_name,Fastq
0,TEST,test_r1.fq.gz
1,LINE2,test_r1.LINE2.fq.gz


In [19]:
len(sample_sheet_single_read.columns)

2

In [20]:
if config_parameters['is_paired'] and len(sample_sheet_single_read.columns) != 3:
    print('do something')

do something


In [ ]:
output_lines.append(config['path_to_samtools']+
                    ' view -b -f 4 -@ '
                    +num_cores+
                    ' -t '+
                    config['fai_file']+
                    ' '+row[0]+'.sam > '+row[0]+'.unmapped.bam'
              

In [66]:



def build(df, config):
    output_info = {}
    for index, row in df.iterrows():
        output_info[row[0]] = {}
        output_info['commands'] = []
        num_cores = str(config_parameters["number_cores"])
        fastq_variable = row[1]
        if config['is_paired']:
            fastq_variable=row[1] +' '+row[2]
        output_info['commands'].append('## Running on 8 threads')
        output_info['commands'].append('## The first step is to align to the genome')
        # The first step is to align the file. How this command is formatted will depend on several parameters
        if config['read_length'] > 69:
            # read length suggests that using bwa mem will be more favorable
            output_info['commands'].append('## Read length suggests that bwa mem is the more favorable option')
            output_info['commands'].append(config['path_to_bwa']+' mem -t '+num_cores+' '+config['path_to_prebuilt_reference']+' '+fastq_variable+' > '+row[0]+'.sam')
        output_info['commands'].append('## Checking alignment stats')
        output_info['commands'].append(config['path_to_samtools']+' stats '+row[0]+'.sam'+' | grep ^SN | cut -f 2- > '+row[0]+'.01sam.align_stats.txt')        
        output_info['commands'].append('## Save unmapped reads and convert them to fastq format for troubleshooting')
        ##
        output_info['commands'].append(config['path_to_samtools']+' view -b -f 4 -@ '+num_cores+' -t '+config['fai_file']+' '+row[0]+'.sam > '+row[0]+'.unmapped.bam')
        output_info['commands'].append(config['path_to_samtools']+' fastq '+row[0]+'.unmapped.bam -1 '+row[0]+'_unmapped1.fastq -2 '+row[0]+'_unmapped2.fastq -s '+row[0]+'_single_read_mapped.fastq')
        ##
        output_info['commands'].append('## Convert to BAM format and remove multimapped reads')
        output_info['commands'].append(config['path_to_samtools']+' view -bq 10 -@ '+num_cores+' '+row[0]+'.sam'+' > '+row[0]+'.uniquelyAligned.bam')
        output_info['commands'].append('## Remove large SAM file and calculate stats for uniquely aligned')
        output_info['commands'].append('rm '+row[0]+'.sam')
        output_info['commands'].append(config['path_to_samtools']+' stats '+row[0]+'.uniquelyAligned.bam | grep ^SN | cut -f 2- > '+row[0]+'.02uniquelyAligned.align_stats.txt')
        output_info['commands'].append('## Now to label the sample inside the bam file, required for Picard Tools')
        output_info['commands'].append(config['path_to_samtools']+' addreplacerg -@ '+num_cores+' -r "@RG\\tID:RG1\\tSM:'+row[0]+'\\tPL:Illumina\\tLB:Library.fa" -o '+row[0]+'.uniquelyAlignedLabeled.bam '+row[0]+'.uniquelyAligned.bam')
        output_info['commands'].append('## Now sort it and index for Picard Tools')
        output_info['commands'].append(config['path_to_samtools']+' sort -@ '+num_cores+' '+row[0]+'.uniquelyAlignedLabeled.bam -o '+row[0]+'.uniquelyAlignedLabeledSorted.bam')
        output_info['commands'].append(config['path_to_samtools']+' index -@ '+num_cores+' '+row[0]+'.uniquelyAlignedLabeledSorted.bam')
        output_info['commands'].append('## Now use PicardTools to mark and remove duplicates')
        output_info['commands'].append('java -jar '+config['path_to_picard_jar']+' MarkDuplicates -REMOVE_DUPLICATES true -VALIDATION_STRINGENCY SILENT -AS true -I '+row[0]+'.uniquelyAlignedLabeledSorted.bam -O '+row[0]+'.rmdup.bam -M '+row[0]+'.picardMetrics.txt')
        output_info['commands'].append('## Run final stats and index the bam file')
        output_info['commands'].append(config['path_to_samtools']+' index -@ '+num_cores+' '+row[0]+'.rmdup.bam')
        output_info['commands'].append(config['path_to_samtools']+' stats '+row[0]+'.rmdup.bam'+' | grep ^SN | cut -f 2- > '+row[0]+'.03rmdup.align_stats.txt')
        #samtools stats RWt_IgG_1.sam | grep ^SN | cut -f 2- > RWt_IgG_1.alignment_stats.01sam.txt
#         for thing in output_lines:
#             print(thing)
#         print()
        output_info['commands'].append(output_lines)
    return output_info
        
    
output_info = build(sample_sheet_2, config_parameters)

In [71]:
def build(df, config):
    output_info = {}
    for index, row in df.iterrows():
        output_commands = []
        num_cores = str(config_parameters["number_cores"])
        fastq_variable = row[1]
        if config['is_paired']:
            fastq_variable=row[1] +' '+row[2]
        output_commands.append('## Running on '+num_cores+' threads')
        output_commands.append('## The first step is to align to the genome')
        # The first step is to align the file. How this command is formatted will depend on several parameters
        if config['read_length'] > 69:
            # read length suggests that using bwa mem will be more favorable
            output_commands.append('## Read length suggests that bwa mem is the more favorable option')
            output_commands.append(config['path_to_bwa']+' mem -t '+num_cores+' '+config['path_to_prebuilt_reference']+' '+fastq_variable+' > '+row[0]+'.sam')
        output_commands.append('## Checking alignment stats')
        output_commands.append(config['path_to_samtools']+' stats '+row[0]+'.sam'+' | grep ^SN | cut -f 2- > '+row[0]+'.01sam.align_stats.txt')        
        output_commands.append('## Save unmapped reads and convert them to fastq format for troubleshooting')
        ##
        output_commands.append(config['path_to_samtools']+' view -b -f 4 -@ '+num_cores+' -t '+config['fai_file']+' '+row[0]+'.sam > '+row[0]+'.unmapped.bam')
        output_commands.append(config['path_to_samtools']+' fastq '+row[0]+'.unmapped.bam -1 '+row[0]+'_unmapped1.fastq -2 '+row[0]+'_unmapped2.fastq -s '+row[0]+'_single_read_mapped.fastq')
        ##
        output_commands.append('## Convert to BAM format and remove multimapped reads')
        output_commands.append(config['path_to_samtools']+' view -bq 10 -@ '+num_cores+' '+row[0]+'.sam'+' > '+row[0]+'.uniquelyAligned.bam')
        output_commands.append('## Remove large SAM file and calculate stats for uniquely aligned')
        output_commands.append('rm '+row[0]+'.sam')
        output_commands.append(config['path_to_samtools']+' stats '+row[0]+'.uniquelyAligned.bam | grep ^SN | cut -f 2- > '+row[0]+'.02uniquelyAligned.align_stats.txt')
        output_commands.append('## Now to label the sample inside the bam file, required for Picard Tools')
        output_commands.append(config['path_to_samtools']+' addreplacerg -@ '+num_cores+' -r "@RG\\tID:RG1\\tSM:'+row[0]+'\\tPL:Illumina\\tLB:Library.fa" -o '+row[0]+'.uniquelyAlignedLabeled.bam '+row[0]+'.uniquelyAligned.bam')
        output_commands.append('## Now sort it and index for Picard Tools')
        output_commands.append(config['path_to_samtools']+' sort -@ '+num_cores+' '+row[0]+'.uniquelyAlignedLabeled.bam -o '+row[0]+'.uniquelyAlignedLabeledSorted.bam')
        output_commands.append(config['path_to_samtools']+' index -@ '+num_cores+' '+row[0]+'.uniquelyAlignedLabeledSorted.bam')
        output_commands.append('## Now use PicardTools to mark and remove duplicates')
        output_commands.append('java -jar '+config['path_to_picard_jar']+' MarkDuplicates -REMOVE_DUPLICATES true -VALIDATION_STRINGENCY SILENT -AS true -I '+row[0]+'.uniquelyAlignedLabeledSorted.bam -O '+row[0]+'.rmdup.bam -M '+row[0]+'.picardMetrics.txt')
        output_commands.append('## Run final stats and index the bam file')
        output_commands.append(config['path_to_samtools']+' index -@ '+num_cores+' '+row[0]+'.rmdup.bam')
        output_commands.append(config['path_to_samtools']+' stats '+row[0]+'.rmdup.bam'+' | grep ^SN | cut -f 2- > '+row[0]+'.03rmdup.align_stats.txt')
        # Now save everything to the output directory
        output_info[row[0]] = output_commands
    return output_info
        
    
output_info = build(sample_sheet_2, config_parameters)

In [72]:
output_info

{'TEST': ['## Running on 8 threads',
  '## The first step is to align to the genome',
  '## Read length suggests that bwa mem is the more favorable option',
  '/home/dwsant/Programs/bwa/bwa mem -t 8 /home/dwsant/Genome_References/MM39_BWA/MM39 test_r1.fq.gz test_r2.fq.gz > TEST.sam',
  '## Checking alignment stats',
  '/home/dwsant/Programs/samtools-1.20/samtools stats TEST.sam | grep ^SN | cut -f 2- > TEST.01sam.align_stats.txt',
  '## Save unmapped reads and convert them to fastq format for troubleshooting',
  '/home/dwsant/Programs/samtools-1.20/samtools view -b -f 4 -@ 8 -t /home/dwsant/Genome_References/MM39_BWA/Mus_musculus.GRCm39.dna.primary_assembly.fa.fai TEST.sam > TEST.unmapped.bam',
  '/home/dwsant/Programs/samtools-1.20/samtools fastq TEST.unmapped.bam -1 TEST_unmapped1.fastq -2 TEST_unmapped2.fastq -s TEST_single_read_mapped.fastq',
  '## Convert to BAM format and remove multimapped reads',
  '/home/dwsant/Programs/samtools-1.20/samtools view -bq 10 -@ 8 TEST.sam > TEST.u

In [73]:
output_info['TEST']

['## Running on 8 threads',
 '## The first step is to align to the genome',
 '## Read length suggests that bwa mem is the more favorable option',
 '/home/dwsant/Programs/bwa/bwa mem -t 8 /home/dwsant/Genome_References/MM39_BWA/MM39 test_r1.fq.gz test_r2.fq.gz > TEST.sam',
 '## Checking alignment stats',
 '/home/dwsant/Programs/samtools-1.20/samtools stats TEST.sam | grep ^SN | cut -f 2- > TEST.01sam.align_stats.txt',
 '## Save unmapped reads and convert them to fastq format for troubleshooting',
 '/home/dwsant/Programs/samtools-1.20/samtools view -b -f 4 -@ 8 -t /home/dwsant/Genome_References/MM39_BWA/Mus_musculus.GRCm39.dna.primary_assembly.fa.fai TEST.sam > TEST.unmapped.bam',
 '/home/dwsant/Programs/samtools-1.20/samtools fastq TEST.unmapped.bam -1 TEST_unmapped1.fastq -2 TEST_unmapped2.fastq -s TEST_single_read_mapped.fastq',
 '## Convert to BAM format and remove multimapped reads',
 '/home/dwsant/Programs/samtools-1.20/samtools view -bq 10 -@ 8 TEST.sam > TEST.uniquelyAligned.bam'

In [74]:
for sample in output_info:
    with open(sample+'_bwa.sh', 'w') as script_file:
        for item in output_info[sample]:
            script_file.write(f"{item}\n")

In [32]:
build(sample_sheet, config_parameters)

['## Running on 8 threads', '## The first step is to align to the genome', '## Read length suggests that bwa mem is the more favorable option', '/home/dwsant/Programs/bwa/bwa mem -t 8 test_r1.fq.gz test_r2.fq.gz > TEST.sam']
['## Running on 8 threads', '## The first step is to align to the genome', '## Read length suggests that bwa mem is the more favorable option', '/home/dwsant/Programs/bwa/bwa mem -t 8 test_r1.LINE2.fq.gz test_r1.LINE2.fq.gz > LINE2.sam']


In [26]:
from copy import deepcopy

In [28]:
config_parameters_single = deepcopy(config_parameters)
config_parameters_single['is_paired'] = False

In [33]:

build(sample_sheet_single_read, config_parameters_single)

['## Running on 8 threads', '## The first step is to align to the genome', '## Read length suggests that bwa mem is the more favorable option', '/home/dwsant/Programs/bwa/bwa mem -t 8 test_r1.fq.gz > TEST.sam']
['## Running on 8 threads', '## The first step is to align to the genome', '## Read length suggests that bwa mem is the more favorable option', '/home/dwsant/Programs/bwa/bwa mem -t 8 test_r1.LINE2.fq.gz > LINE2.sam']


In [15]:
config_parameters

{'read_length': 150,
 'is_paired': True,
 'path_to_bwa': '/home/dwsant/Programs/bwa/bwa',
 'number_cores': 8,
 'path_to_prebuilt_reference': '/home/dwsant/Genome_References/MM39_BWA/MM39',
 'path_to_samtools': '/home/dwsant/Programs/samtools-1.20/samtools',
 'path_to_picard_jar': '/home/dwsant/Programs/picard/build/libs/picard.jar',
 'fai_file': '/home/dwsant/Genome_References/MM39_BWA/Mus_musculus.GRCm39.dna.primary_assembly.fa.fai'}

In [17]:
list_of_args = ['This should be line 1', 'this should be line 2', 'and this should be line 3.']


with open('your_file.txt', 'w') as script_file:
    for item in list_of_args:
        #script_file.write(item, 'a')
        script_file.write(f"{item}\n")
#with open("test.txt") as 
#with open("test.txt") as myfile: myfile.write("appended text",'a')